In [1]:
import glob, cv2
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
tf.config.list_physical_devices('GPU')

2025-09-01 15:41:42.798399: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-09-01 15:41:42.808970: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-09-01 15:41:42.812850: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
dogs = glob.glob("cat_and_dog/Dog/*")
cats = glob.glob("cat_and_dog/Cat/*")

In [3]:
df = pd.concat([
    pd.DataFrame({"label":1,"path":dogs}),
    pd.DataFrame({"label":0,"path":cats})
], ignore_index = True).sample(frac=1, random_state=42).reset_index(drop=True)

In [4]:
def load_image(path, img_size):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, img_size)
    return img

In [5]:
class MyGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, function, batch_size = 32,
                 image_size = (224, 224), shuffle = True):
        
        self.df = df
        self.batch_size = batch_size
        self.img_size = image_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.df))
        self.on_epoch_end()
        self.function = function
    # 배치를 몇번 반복할지 반환
    def __len__(self):
        return int(np.ceil(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[
            index*self.batch_size : (index+1)*self.batch_size]
        batch_df = self.df.iloc[batch_indices]
        
        batch_images = np.array([self.function(
            p, self.img_size) for p in batch_df['path']])
        batch_labels = np.array(batch_df['label'])
        
        return batch_images, batch_labels
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

In [6]:
train_df, val_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df['label'])
test_df, val_df = train_test_split(
    val_df, test_size=0.5, random_state=42, stratify=val_df['label'])
train_gen = MyGenerator(train_df,load_image, batch_size=32)
val_gen = MyGenerator(val_df, load_image, batch_size=32)

In [7]:
backbone = tf.keras.applications.MobileNetV2(
    input_shape = (224,224,3), include_top = False)
backbone.trainable = False

inputs = tf.keras.layers.Input(shape=(224,224,3))
x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
x = backbone(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

2025-09-01 15:41:42.971589: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-01 15:41:42.972719: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-09-01 15:41:42.976614: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-09-01 15:41:42.979370: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [8]:
model = tf.keras.Model(inputs, outputs)

In [10]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.BinaryCrossentropy()
metrics = [tf.keras.metrics.BinaryAccuracy()]
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    workers=4,                 # CPU 코어 수에 맞춰 조정
    use_multiprocessing=True,  # 병렬 처리 활성화
)

Epoch 1/10
 45/586 [=>............................] - ETA: 1:24 - loss: 0.0098 - binary_accuracy: 0.9972    

Process Keras_worker_ForkPoolWorker-80:
Process Keras_worker_ForkPoolWorker-76:
Process Keras_worker_ForkPoolWorker-74:
Process Keras_worker_ForkPoolWorker-75:
Process Keras_worker_ForkPoolWorker-77:
Process Keras_worker_ForkPoolWorker-78:
Process Keras_worker_ForkPoolWorker-79:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/conda/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/conda/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/conda/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/conda/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/conda/lib/python3.9/multiprocessing/process.py", line 315, in _b